# Least Squares Support Vector Machine

This notebook contains the LSSVM implementation based on the one implemented by *Suman Duttaa, Mandeep Singha, and Amod Kumar* in [this paper](https://www.sciencedirect.com/science/article/pii/S1746809418300478?casa_token=O4FQDvOwuqEAAAAA:0KLqq4okPDih95kJeWcOxLSZawKwH9OgsLjCxOS2joy_1cR5T28s6M0IVV17-DslhU9C-yWWDb0). The pre-processing and feature extraction for the same can be found [here](https://github.com/Parthiv-M/eeg-classification/blob/main/dutta_2018/Feature%20Extraction.ipynb).

In [ ]:
import numpy as np
from numpy.linalg import eig, eigvals
import pandas as pd
from numpy import dot, exp
from scipy.spatial.distance import cdist
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import MinMaxScaler

In [ ]:
def get_kernel(name, **params):
    """The method that returns the kernel function, given the 'kernel'
    parameter.
    """

    def linear(x_i, x_j):
        return dot(x_i, x_j.T)

    def poly(x_i, x_j, d=params.get('d', 3)):
        return (dot(x_i, x_j.T) + 1) ** d

    def rbf(x_i, x_j, sigma=params.get('sigma', 1)):
        return exp(-cdist(x_i, x_j) ** 2 / sigma ** 2)

    kernels = {'linear': linear, 'poly': poly, 'rbf': rbf}

    if kernels.get(name) is None:
        raise KeyError(
            f"Kernel '{name}' is not defined, try one in the list: "
            f"{list(kernels.keys())}."
        )
    else:
        return kernels[name]

In [ ]:
import codecs
import json

def dump_model(model_dict, file_encoder, filepath='model'):
    with open(f"{filepath.replace('.json', '')}.json", 'w') as fp:
        json.dump(model_dict, fp, default=file_encoder)

def load_model(filepath='model'):
    helper_filepath = filepath if filepath.endswith('.json') else f"{filepath}.json"
    file_text = codecs.open(helper_filepath, 'r', encoding='utf-8').read()
    model_json = json.loads(file_text)

    return model_json

In [ ]:
def dummie2multilabel(X):
    """Convert dummies to multilabel"""
    N = len(X)
    X_multi = np.zeros((N,1),dtype='int')
    for i in range(N):
        temp = np.where(X[i]==1)[0] # find 1 in the array
        if temp.size == 0: # is a empty array, there is no '1' in the X[i] array
            X_multi[i] = 0 # so we denote this class '0'
        else:
            X_multi[i] = temp[0] + 1
    return X_multi.T[0]

In [ ]:
def numpy_json_encoder(obj):
    if type(obj).__module__ == np.__name__:
        if isinstance(obj, np.ndarray):
            return obj.tolist()
        else:
            return obj.item()
    raise TypeError(f"""Unable to  "jsonify" object of type :', {type(obj)}""")

In [ ]:
class LSSVC(): 
    def __init__(self, gamma=1, kernel='rbf', **kernel_params): 
        # Hyperparameters
        self.gamma = gamma
        self.kernel_ = kernel
        self.kernel_params = kernel_params
        
        # Model parameters
        self.alpha = None
        self.b = None
        self.sv_x = None
        self.sv_y = None
        self.y_labels = None
        
        self.K = get_kernel(kernel, **kernel_params)
    
    def _optimize_parameters(self, X, y_values):
        """Help function that optimizes the dual variables through the 
        use of the kernel matrix pseudo-inverse.
        """
        sigma = np.multiply(y_values*y_values.T, self.K(X,X))
        
        A = np.block([
            [0, y_values.T],
            [y_values, sigma + self.gamma**-1 * np.eye(len(y_values))]
        ])
        B = np.array([0]+[1]*len(y_values))
        
        A_cross = np.linalg.pinv(A)

        solution = dot(A_cross, B)
        b = solution[0]
        alpha = solution[1:]
        
        return (b, alpha)
    
    def fit(self, X, y):
        """Fits the model given the set of X attribute vectors and y labels.
        - X: ndarray of shape (n_samples, n_attributes)
        - y: ndarray of shape (n_samples,) or (n_samples, n)
            If the label is represented by an array of n elements, the y 
            parameter must have n columns.
        """
        y_reshaped = y.reshape(-1,1) if y.ndim==1 else y

        self.sv_x = X
        self.sv_y = y_reshaped
        self.y_labels = np.unique(y_reshaped, axis=0)
        
        if len(self.y_labels) == 2: # binary classification
            # converting to -1/+1
            y_values = np.where(
                (y_reshaped == self.y_labels[0]).all(axis=1)
                ,-1,+1)[:,np.newaxis] # making it a column vector
            
            self.b, self.alpha = self._optimize_parameters(X, y_values)
        
        else: # multiclass classification, one-vs-all approach
            n_classes = len(self.y_labels)
            self.b = np.zeros(n_classes)
            self.alpha = np.zeros((n_classes, len(y_reshaped)))
            
            for i in range(n_classes):
                # converting to +1 for the desired class and -1 for all 
                # other classes
                y_values = np.where(
                    (y_reshaped == self.y_labels[i]).all(axis=1)
                    ,+1,-1)[:,np.newaxis]
  
                self.b[i], self.alpha[i] = self._optimize_parameters(X, y_values)
        
    def predict(self, X):
        """Predicts the labels of data X given a trained model.
        - X: ndarray of shape (n_samples, n_attributes)
        """
        if self.alpha is None:
            raise Exception(
                "The model doesn't see to be fitted, try running .fit() method first"
            )

        X_reshaped = X.reshape(1,-1) if X.ndim==1 else X
        KxX = self.K(self.sv_x, X_reshaped)
        
        if len(self.y_labels)==2: # binary classification
            y_values = np.where(
                (self.sv_y == self.y_labels[0]).all(axis=1),
                -1,+1)[:,np.newaxis]

            y = np.sign(dot(np.multiply(self.alpha, y_values.flatten()), KxX) + self.b)
            
            y_pred_labels = np.where(y==-1, self.y_labels[0], self.y_labels[1])
        
        else: # multiclass classification, one-vs-all approach
            y = np.zeros((len(self.y_labels), len(X)))
            for i in range(len(self.y_labels)):
                y_values = np.where(
                    (self.sv_y == self.y_labels[i]).all(axis=1),
                    +1, -1)[:,np.newaxis]
                y[i] = dot(np.multiply(self.alpha[i], y_values.flatten()), KxX) + self.b[i]
            
            predictions = np.argmax(y, axis=0)
            y_pred_labels = np.array([self.y_labels[i] for i in predictions])
            
        return y_pred_labels

    def dump(self, filepath='model', only_hyperparams=False):
        """This method saves the model in a JSON format.
        - filepath: string, default = 'model'
            File path to save the model's json.
        - only_hyperparams: boolean, default = False
            To either save only the model's hyperparameters or not, it 
            only affects trained/fitted models.
        """
        model_json = {
            'type': 'LSSVC',
            'hyperparameters': {
                'gamma': self.gamma,
                'kernel': self.kernel_,
                'kernel_params': self.kernel_params
            }           
        }

        if (self.alpha is not None) and (not only_hyperparams):
            model_json['parameters'] = {
                'alpha': self.alpha,
                'b': self.b,
                'sv_x': self.sv_x,
                'sv_y': self.sv_y,
                'y_labels': self.y_labels
            }
        
        dump_model(model_dict=model_json, file_encoder=numpy_json_encoder, filepath=filepath)
        
    @classmethod
    def load(cls, filepath, only_hyperparams=False):
        """This class method loads a model from a .json file.
        - filepath: string
            The model's .json file path.
        - only_hyperparams: boolean, default = False
            To either load only the model's hyperparameters or not, it 
            only has effects when the dump of the model as done with the
            model's parameters.
        """
        model_json = load_model(filepath=filepath)

        if model_json['type'] != 'LSSVC':
            raise Exception(
                f"Model type '{model_json['type']}' doesn't match 'LSSVC'"
            )

        lssvc = LSSVC(
            gamma = model_json['hyperparameters']['gamma'],
            kernel = model_json['hyperparameters']['kernel'],
            **model_json['hyperparameters']['kernel_params']
        )

        if (model_json.get('parameters') is not None) and (not only_hyperparams):
            lssvc.alpha = np.array(model_json['parameters']['alpha'])
            lssvc.b = np.array(model_json['parameters']['b'])
            lssvc.sv_x = np.array(model_json['parameters']['sv_x'])
            lssvc.sv_y = np.array(model_json['parameters']['sv_y'])
            lssvc.y_labels = np.array(model_json['parameters']['y_labels'])

        return lssvc

In [ ]:
base_data_csv = pd.read_csv('letter_base.csv')

In [ ]:
idx = 0
data = {
    'task': []
}
coeff_mat = base_data_csv.iloc[:,2:]
while idx < len(coeff_mat):
    # select set of six vectors according to channel
    set_of_six = base_data_csv.iloc[idx:idx+6,2:]
    tasks = base_data_csv['task'][idx:idx+6].to_numpy()
    # calculate the covariance matrix of the six vectors
    cov_mat = np.cov(set_of_six)
    if cov_mat.size == 36:
        # calculate the eigen values of the covariance matrix
        w = eigvals(cov_mat)
        # create a csv for easier usability
        for ind, task in enumerate(tasks):
            data['task'].append(tasks[ind])
            for val in w:
                if ind in data.keys():
                    data[ind].append(w[ind].real)
                else:
                    data[ind] = [w[ind].real]
    idx += 6

In [ ]:
dataframe = pd.DataFrame(data)
# convert the dataframe to csv
dataframe.to_csv('letter_base_eigen.csv', encoding='utf-8')
print('letter_base_eigen.csv created')

In [ ]:
data_csv = pd.read_csv('letter_base_eigen.csv')

In [ ]:
X = data_csv.iloc[:,2:]
y_string = data_csv['task'].to_numpy()
y_list = []
for task in y_string:
    if task == 'baseline':
        y_list.append(0)
    elif task == 'letter-composing':
        y_list.append(1)
y = np.array(y_list)

# create the train test data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=2020)

# Scaling features (from sklearn)
scaler = MinMaxScaler()
scaler.fit(X_train)
X_tr_norm = scaler.transform(X_train)
X_ts_norm = scaler.transform(X_test)

# Get information about input and outputs
print(f"X_train.shape: {X_train.shape}")
print(f"X_test.shape:  {X_test.shape}")
print(f"y_train.shape: {y_train.shape}")
print(f"y_test.shape:  {y_test.shape}")
print(f"np.unique(y_train): {np.unique(y_train)}")
print(f"np.unique(y_test):  {np.unique(y_test)}")

In [ ]:
# Use the classifier with different kernels

print('Gaussian kernel:')
lssvc = LSSVC(gamma=1, kernel='rbf', sigma=.5) # Class instantiation
lssvc.fit(X_tr_norm, y_train) # Fitting the model
y_pred = lssvc.predict(X_ts_norm) # Making predictions with the trained model
acc = accuracy_score(dummie2multilabel(y_test), dummie2multilabel(y_pred)) # Calculate Accuracy
print('acc_test = ', acc, '\n')

print('Polynomial kernel:')
lssvc = LSSVC(gamma=1, kernel='poly', d=2)
lssvc.fit(X_tr_norm, y_train)
y_pred = lssvc.predict(X_ts_norm)
acc = accuracy_score(dummie2multilabel(y_test), dummie2multilabel(y_pred))
print('acc_test = ', acc, '\n')

print('Linear kernel:')
lssvc = LSSVC(gamma=1, kernel='linear')
lssvc.fit(X_tr_norm, y_train)
y_pred = lssvc.predict(X_ts_norm)
acc = accuracy_score(dummie2multilabel(y_test), dummie2multilabel(y_pred))
print('acc_test = ', acc, '\n')